In [1]:
%load_ext autoreload
%autoreload 2
import nltk
nltk.download('stopwords')
from graphrank.graphrank import GraphRank
from graphrank.utils import GraphUtils, TextPreprocess
from graphrank import dgraph as dg
from graphrank import build_meeting_graph as meet_graph
import networkx as nx
import text_preprocessing.preprocess as tp
import datetime
import json
import time

import pydgraph

[nltk_data] Downloading package stopwords to /home/ray/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ray/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [39]:
json_file = 'meeting.json'
meet_data = meet_graph.read_json(json_file)
mg = meet_graph.build_meeting_graph(json_file)

['6baa3490-69d6-48fc-b5d4-3994e3e8fae0',
 'b34997',
 None,
 '40c778dca28f49f7aa00ea374019d3bb',
 '439f72efc988422fb0c35e3c1316b150',
 'b4b7d184ef2e4bc28216677d64a901b0',
 '4c491448e1eb495abeac36e708622034',
 'e42e7760d9e0468989f6c4c3af161adc',
 '25d67d06a3ee454fa38fb9c641887e69',
 '130d727e79e6475f9ee3e2a0b50ec415',
 '0d7db29d70e94dfeacf5c8d61ae624c3',
 'd3f32e66ee2941248f17de3cb80c809d',
 '7fec55036f4647e1ad149588d76641b7',
 'd191dd997eaf4dcd9091a3bfb941a9a9',
 'd0a1a5f5a4224a54ab6388f01d795f5a',
 'cfc55ab1014546738e6b33cccbd08b31',
 '6de739b0ba144273b60f83782aae7759',
 '2f5a411fd7b5473aa177ea88c68e9dfd',
 '6b68d50bd86f4c9893c5bf6f361152d6',
 '51018cf965dc453098d5329dea6e3d6f',
 '284ce146416b4036a11a71fe3fc8ce82',
 '75d0b6f3a2fc48f8ba5a3bd330b8a6f8',
 '7c5b273ac9994a44875cfa3322e689e0',
 'b7cf9411c58347f5b442a674f4769526',
 '559896253c6b4069877d88697aa909ed',
 '82f0f2cae0ef45349930dcb20c76efd9',
 'bb0f8d96164b48dd99d7b1dd5d25ef33',
 '6401b3dab57743c38104907bf91d8e7e',
 '0925e07bf7b34b

In [28]:

def create_client_stub():
    #return pydgraph.DgraphClientStub('111.93.155.194:9080')
    return pydgraph.DgraphClientStub('localhost:9080')


def create_client(client_stub):
    return pydgraph.DgraphClient(client_stub)


def drop_all(client):
    return client.alter(pydgraph.Operation(drop_all=True))


def set_schema(client):
    schema = """
    contextId: string @index(exact) .
    contextInstanceId: string @index(exact).
    contextInstanceIds: uid .
    segments: uid .
    segmentId: string @index(exact) .
    keyword: string @index(exact) .
    name: string @index(exact) .
    meetingid: string @index(exact) .
    score: int .
    neighbour: uid @reverse .
    user.id: string @index(exact) .
    """
    return client.alter(pydgraph.Operation(schema=schema))

    

def build_custom_graph(input_text, window=2):
    graph = nx.Graph()
    
    graph.add_nodes_from([word[0] for word in input_text])
    
    
    for i, (node1, pos) in enumerate(input_text):
        if not pos:
            continue

        for j in range(i + 1, min(i + window, len(input_text))):
            node2, pos2 = input_text[j]
            if pos2 and node1 != node2:
                graph.add_edge(node1, node2)
    return graph


def write_node(client, node, contextInstanceId, contextInstanceId_uid, position, seg_position):
    txn = client.txn()
    global initial 
    try:
        # The below query checks if a keyword name exist in the graph from the filtered subgraph of same context Instance Id ie. same meeting
        
        query = """query all($a: string, $b: string) {
                    all(func: eq(name, $a)) @filter(eq(contextInstanceId, $b)){
                            uid
                        }
                    }"""
    
        variables = {'$a': node, '$b': contextInstanceId }
        res = client.query(query, variables=variables)
        ppl = json.loads(res.json)

        # IF keyword node doesn't exist, create the node.
        
        if len(ppl['all'])==0:
            data = {
                'uid' : '_:' + node,
                'name' : node,
                'contextInstanceId' : contextInstanceId
            }


            assigned = txn.mutate(set_obj=data,commit_now=True)
            txn = client.txn()
            
            for i in assigned.uids:
                keyword_uid = assigned.uids[i]


            # IF creating first node, create a link btw first keyword & ContextInstanceId
            #if position==0 and seg_position==1:
            if initial == True:
                data = {
                    'uid' : contextInstanceId_uid,
                    'wordGraph' : {
                        'uid' : keyword_uid
                    }
                }
                assigned = txn.mutate(set_obj=data,commit_now=True)
                txn = client.txn()
                initial = False

    except Exception as e:
        
            print('Error: {}'.format(e))
    txn.commit()
    return True


def write_edge(client, node, contextInstanceId, contextInstanceId_uid, text):
    txn = client.txn()
    
    try:
        # The below query gets the UID of the Node A where a relationship needs to be drawn btw (Node A, Node B)

        query = """query all($a: string, $b: string) {
                       all(func: eq(name, $a)) @filter(eq(contextInstanceId, $b)){
                           uid
                       }
                }"""

        variables = {'$a': node[0], '$b': contextInstanceId }
        res = client.query(query, variables=variables)
        ppl = json.loads(res.json)
        #print (ppl)
        
        node_left = ppl['all'][0]['uid']

        # The below query gets the UID of the Node B where a relationship needs to be drawn btw (Node A, Node B)

        query = """query all($a: string, $b: string) {
                       all(func: eq(name, $a)) @filter(eq(contextInstanceId, $b)){
                           uid
                       }
                }"""

        variables = {'$a': node[1], '$b': contextInstanceId }
        res = client.query(query, variables=variables)
        ppl = json.loads(res.json)


        node_right = ppl['all'][0]['uid']

        # After recieving both the uid of Node A & Node B, draw the relationship btw them.
        
        data = {
            'uid': node_left,
            'neighbour': [
                {
                    'uid': node_right
                }
            ]
        }


        assigned = txn.mutate(set_obj=data,commit_now=True)
        txn = client.txn()

    except Exception as e:
            #print (text)
            #print (node[0])
            print('Error: {}'.format(e))
    txn.commit()
    return True

def update_graph(client, mg):
    graph_obj = list(mg.edges.data())
    contextId = graph_obj[0][0]
    contextInstanceId = graph_obj[0][1]
    
    nodes_count = 0
    edges_count = 0
    segments_count = 0
    global initial
    initial = True
    txn = client.txn()

    # The below query takes in the contextId and does a indexed search on the graph to get the Unique Integer Id (uid)
    # eq function checks for equality on the indexed node.

    query = """query all($a: string) {
                   all(func: eq(contextId, $a)) {
                       uid
                    }
               }"""

    variables = {'$a': contextId }
    res = client.query(query, variables=variables)  # client.query() operation is executed on a local copy of the graph.
    ppl = json.loads(res.json)


    # The below code checks if the result recieved from the previous query contains the uid, if it does, it initializes to a local var.
    if len(ppl['all'])>0 :
            contextId_uid = ppl['all'][0]['uid']

    # If the query doesn't have uid, ie. the instanceId doesn't exist in the graph, we create a new InstanceId.

    else:
        print ("Creating new ContextId")

        data = {
            'uid': '_:' + contextId,     # <underscore><colon><Identifier> '_:identifier' is the RDF format to create a new uid.
            'contextId' : contextId,            
        }

        # The mutate operation executes an update query on the graph DB.
        assigned = txn.mutate(set_obj=data,commit_now=True)
        txn = client.txn()

        # Get the UID of the recently created Node, ie. contextId.
        for i in assigned.uids:
            contextId_uid = assigned.uids[i]

        print ("Context ID: ", contextId_uid)


        # Follow the same methodology on contextInstanceId, ie. Create if it does not exist.

        query = """query all($a: string) {
                       all(func: eq(contextInstanceId, $a)) {
                           uid
                       }
                }"""

        variables = {'$a': contextInstanceId }
        res = client.query(query, variables=variables)
        ppl = json.loads(res.json)

        if len(ppl['all'])>0 :
            contextInstanceId_uid = ppl['all'][0]['uid']

        else:
            print ("Creating new ContextInstanceId")

            data = {
                'uid': '_:' + contextInstanceId, 
                'contextInstanceId' : contextInstanceId
            }

            assigned = txn.mutate(set_obj=data,commit_now=True)
            txn = client.txn()

            for i in assigned.uids:
                contextInstanceId_uid = assigned.uids[i]

            print ("Creating a relation between context ID and Context Instance Id")

            # The below code creates a realationship (edge or predicate) from context Id to Context Instance Id

            data = {
                'uid' : contextId_uid,
                'contextInstanceIds':[
                    {
                        'uid' : contextInstanceId_uid
                    }
                ]
            }

            assigned = txn.mutate(set_obj=data,commit_now=True)
            txn = client.txn()

        print ("Context Instance ID: ", contextInstanceId_uid)

        print ("\nCreating Segments: ")

        # Get all the contents from the graph_obj including segments, users & their properties.

        for seg_position, seg in enumerate(graph_obj[1:]):


            # Iterate through the segments and compute word graph & the segments properties.
            if ('rel' in seg[2].keys() ):
                if seg[2]['rel']=="has_segment":
                    print ("Processing Segment " + str(seg_position) )
                    transcribedBy = ""
                    segments_count+=1
                    segmentId = seg[1]
                    check1, check2 = False, False
                    for s in graph_obj:
                        if 'rel' in seg[2].keys() and s[2]['rel']=="transcribed_by"  and s[0]==seg[1]:
                            transcribedBy = s[1]
                            check1 = True
                        if 'rel' in seg[2].keys() and s[2]['rel']=="spoken_by" and s[0]==seg[1]:
                            spokenBy = s[1]
                            #print (spokenBy)
                            check2 = True
                        if check1 and check2:
                            break

                    # Create User Node for each Segment
                    
                    query = """query all($a: string) {
                               all(func: eq(user.id, $a)) {
                                   uid
                               }
                            }"""
  
                    variables = {'$a': spokenBy }
                    res = client.query(query, variables=variables)
                    ppl = json.loads(res.json)
                    print (ppl)
                    if len(ppl['all'])>0 :
                        user_uid = ppl['all'][0]['uid']
                        print (spokenBy, user_uid)
                    
                    else:
                        data = {
                                'uid' : '_:' + spokenBy,
                                'user.id' : spokenBy
                            }

                        assigned = txn.mutate(set_obj=data,commit_now=True)
                        txn = client.txn()

                        for i in assigned.uids:
                            user_uid = assigned.uids[i]

                    # Create Segment Node
                    #print (spokenBy)
                    #print (user_uid)
                    data = {
                        'uid': '_:' + segmentId, 
                        'segmentId' : segmentId,
                        'transcriber' : transcribedBy,
                        'spokenBy' : {
                            'uid' : user_uid
                        }
                    }

                    assigned = txn.mutate(set_obj=data,commit_now=True)
                    txn = client.txn()

                    for i in assigned.uids:
                        segmentId_uid = assigned.uids[i]

                    # draw a realationship (edge or predicate) from Context Instance ID to Segment Node

                    data = {
                        'uid' : contextInstanceId_uid,
                        'segmentIds':[
                            {
                                'uid' : segmentId_uid
                            }
                        ]
                    }


                    assigned = txn.mutate(set_obj=data,commit_now=True)
                    txn = client.txn()


                    # The below code computes nodes and edges for each segment which are iterated and extends the Dgraph on every Iter

                    if len(seg[2]['text'])!=0:

                        # Text Preprocessing using custom Package
                        text_sent, text_pos = tp.preprocess(seg[2]['text'], stop_words=False, lemma=False, pos=True)
                        pos_text = [(word,pos) for sent in text_pos for word, pos in sent]

                        # Building custom Nx Graph and returning a object 
                        g = build_custom_graph(pos_text,window=2)
                        nodes = list(dict(g.nodes()).keys())
                        edges = list(dict(g.edges()).keys())
                        nodes_count+=len(nodes)
                        edges_count+=len(edges)*2

                        # Create node w/ properties for everykeyword in the segment.
                        for position, node in enumerate(nodes):
                            write_node(client, node, contextInstanceId, contextInstanceId_uid, position, seg_position)

                        # Create relationship between Nodes ie. Keywords.
                        for edge in edges:
                            write_edge(client, edge, contextInstanceId, contextInstanceId_uid, seg[2]['text'])


    # commit the local ref graph to the Dgraph daemon

    txn.commit()
    print ("Nodes Count: ", nodes_count, "Edges Count: ", edges_count, "Segments Count: ", segments_count)
    
    
if __name__ == '__main__':
    try:
        # Get Meeting level graph object.
        json_file = 'meeting.json'
        meet_data = meet_graph.read_json(json_file)
        mg = meet_graph.build_meeting_graph(json_file)
        
        start = time.process_time()
        start_nanosec = time.time()
        client_stub = create_client_stub()
        client = create_client(client_stub)
    
        # remove any existing data (for testing purposes)
        drop_all(client)
        set_schema(client)
        
        update_graph(client, mg)
        end = time.process_time()
        end_nanosec = time.time()
    
        print('\nDONE!\n')
        print ('Execution time (Fraction seconds): ' + str(end - start))
        print ('Execution time (nano seconds): ' + str(end_nanosec - start_nanosec))
    except Exception as e:
        print('Error: {}'.format(e))


Creating new ContextId
Context ID:  0x13a01
Creating new ContextInstanceId
Creating a relation between context ID and Context Instance Id
Context Instance ID:  0x13a02

Creating Segments: 
Processing Segment 1
{'all': []}
Processing Segment 2
{'all': []}
Processing Segment 3
{'all': [{'uid': '0x13a05'}]}
394d6dd0-7dbe-4d77-92c6-2af7f8314329 0x13a05
Processing Segment 4
{'all': [{'uid': '0x13a03'}]}
7eeb4d50-dba0-4cf8-a393-0d93a5ad2c69 0x13a03
Processing Segment 5
{'all': [{'uid': '0x13a05'}]}
394d6dd0-7dbe-4d77-92c6-2af7f8314329 0x13a05
Processing Segment 6
{'all': [{'uid': '0x13a05'}]}
394d6dd0-7dbe-4d77-92c6-2af7f8314329 0x13a05
Processing Segment 7
{'all': [{'uid': '0x13a03'}]}
7eeb4d50-dba0-4cf8-a393-0d93a5ad2c69 0x13a03
Processing Segment 8
{'all': [{'uid': '0x13a05'}]}
394d6dd0-7dbe-4d77-92c6-2af7f8314329 0x13a05
Processing Segment 9
{'all': [{'uid': '0x13a03'}]}
7eeb4d50-dba0-4cf8-a393-0d93a5ad2c69 0x13a03
Processing Segment 10
{'all': [{'uid': '0x13a05'}]}
394d6dd0-7dbe-4d77-92

In [37]:
user = []
for seg in list(mg.edges.data())[1:]:
    if seg[2]['rel'] == "spoken_by":
        user.append(seg[1])
set(user)

{'394d6dd0-7dbe-4d77-92c6-2af7f8314329',
 '7eeb4d50-dba0-4cf8-a393-0d93a5ad2c69'}

In [ ]:
client_stub = pydgraph.DgraphClientStub('localhost:9080')
client = pydgraph.DgraphClient(client_stub)
contextInstanceId = "b34997"
    
query = """query all($a: string, $b: string) {
    all(func: eq(name, $a)) @filter(eq(contextInstanceId, $b)){
        uid
    }
}"""

node = 'hey'
variables = {'$a': node, '$b': contextInstanceId }
res = client.query(query, variables=variables)
ppl = json.loads(res.json)

In [ ]:
len(ppl['all'])

In [ ]:
graph_obj = list(mg.edges.data())
count = 0
for i in graph_obj:
    if i[2]['rel']=="has_segment":
        count+=1
print(count)

In [ ]:
for i in graph_obj:
    print (i)
    if i[2]['rel']=="has_segment":
        text_sent, text_pos = tp.preprocess(i[2]['text'], stop_words=False, lemma=False, pos=True)
        pos_text = [(word,pos) for sent in text_pos for word, pos in sent]
        g = build_custom_graph(pos_text,window=2)
        nodes = list(dict(g.nodes()).keys())
        edges = list(dict(g.edges()).keys())
        #print (i)

In [ ]:
json_file = 'meeting.json'
meet_data = meet_graph.read_json(json_file)
mg = meet_graph.build_meeting_graph(json_file)
#[l for l in list(mg.edges.data()) if l[2]['rel']!="has_meeting"]
obj = list(mg.edges.data())
d = '439f72efc988422fb0c35e3c1316b150'
for seg in obj:
    if seg[2]['rel']=="spoken_by"  and seg[0]==d:
        print (seg)

In [12]:
query = """query all($a: string, $b: string) {
    all(func: eq(name, $a)) @filter(eq(contextInstanceId, $b)){
        uid
    }
}"""
node = "scope"
contextInstanceId = "b34997"
variables = {'$a': node, '$b': contextInstanceId }
res = client.query(query, variables=variables)
ppl = json.loads(res.json)

ppl

{'all': [{'uid': '0x11e57'}]}

In [13]:
client_stub = pydgraph.DgraphClientStub('localhost:9080')
client = pydgraph.DgraphClient(client_stub)
contextInstanceId = "b34997"

query = """query all($a: string) {
                   all(func: eq(contextInstanceId, $a)) {
                       uid
                   }
            }"""
    
variables = {'$a': contextInstanceId }
res = client.query(query, variables=variables)
ppl = json.loads(res.json)
    
if len(ppl['all'])>0 :
    contextInstanceId_uid = ppl['all'][0]['uid']

query = """ query all($a: string){
                find(func: uid($a)) @recurse{
                expand(_all_){
                    expand(_all_)
              }
            }
            }
        """

#node = 'hey'
#variables = {'$a': node, '$b': contextInstanceId }
variable = {'$a': contextInstanceId_uid}
res = client.query(query, variables=variable)
ppl = json.loads(res.json)
ppl

{'find': [{'wordGraph': [{'name': 'Hello',
     'neighbour': [{'name': 'yeah',
       'neighbour': [{'name': 'hi',
         'neighbour': [{'name': '.', 'contextInstanceId': 'b34997'}],
         'contextInstanceId': 'b34997',
         '~neighbour': [{'contextInstanceId': 'b34997', 'name': 'yeah'},
          {'contextInstanceId': 'b34997', 'name': 'it'},
          {'contextInstanceId': 'b34997', 'name': 'work'}]},
        {'name': '.', 'contextInstanceId': 'b34997'},
        {'name': 'I',
         'neighbour': [{'name': 'yeah', 'contextInstanceId': 'b34997'},
          {'name': '.', 'contextInstanceId': 'b34997'},
          {'name': 'can', 'contextInstanceId': 'b34997'},
          {'name': 'think', 'contextInstanceId': 'b34997'},
          {'name': 'just', 'contextInstanceId': 'b34997'},
          {'name': 'on', 'contextInstanceId': 'b34997'},
          {'name': 'the', 'contextInstanceId': 'b34997'},
          {'name': 'a',
           'neighbour': [{'name': 'I', 'contextInstanceId': 'b34

In [11]:
client_stub = pydgraph.DgraphClientStub('localhost:9080')
client = pydgraph.DgraphClient(client_stub)

query = """query all($a: string) {
                               all(func: eq(user.id, $a)) {
                                   uid
                               }
                            }"""
  
variables = {'$a': "394d6dd0-7dbe-4d77-92c6-2af7f8314329" }
res = client.query(query, variables=variables)
ppl = json.loads(res.json)
if len(ppl['all'])>0 :
    user_uid = ppl['all'][0]['uid']
    print (user_uid)


0x13069


In [27]:
##testing

client_stub = pydgraph.DgraphClientStub('localhost:9080')
client = pydgraph.DgraphClient(client_stub)
txn = client.txn()
data = {
        'uid' : '_:' + "user1",
        'user.id' : "user1"
    }

assigned = txn.mutate(set_obj=data,commit_now=True)

for i in assigned.uids:
    user_uid = assigned.uids[i]
    print (user_uid)
#txn.commit()
#txn = client.txn()
query = """query all($a: string) {
                               all(func: eq(user.id, $a)) {
                                   uid
                               }
                            }"""
  
variables = {'$a': "user1" }
res = client.query(query, variables=variables)
ppl = json.loads(res.json)
print (ppl)
# if len(ppl['all'])>0 :
#     user_uid = ppl['all'][0]['uid']
#     print (user_uid)
txn.commit()

0x13a00
{'all': [{'uid': '0x139fc'}, {'uid': '0x139fd'}, {'uid': '0x139fe'}, {'uid': '0x139ff'}, {'uid': '0x13a00'}]}


Exception: Transaction has already been committed or discarded

In [38]:
graph_obj

NameError: name 'graph_obj' is not defined